# Experiment 2: K-Means using SIF-weighted fastText embeddings

In this experiment, summaries are generated by running K-Means clustering on the emedded sentences of a document. The length of the summary is determined by the number of clusters *k*, where *k* equals to the desired number of sentences in the summary.
Sentence embeddings are obtained as the average of the individual fastText word embeddings, weighted by the smooth inverse frequencies.

In [ ]:
import pandas as pd
import tqdm
from rouge import Rouge

In [ ]:
from Fasttext import FTEmbedder
from Preprocessors import StandardPreprocessor
from Evaluator import USEevaluator
from models.unsupervised import kMeans

In [ ]:
test_data = pd.read_pickle("./training_data/test_raw.pkl")

In [ ]:
test_data = test_data.sort_values(by=['Language'])

In [ ]:
test_data.head()

In [ ]:
summarizer = kMeans(FTEmbedder, StandardPreprocessor)
comparator = USEevaluator(metric="cosine")

In [ ]:
summaries = []
cosims = []

In [ ]:
flatdict = {}
rouge = Rouge()

In [ ]:
for i, row in tqdm.tqdm_notebook(test_data.iterrows(), total=len(test_data.index)):
    try:
        smry = summarizer.summarize(row.Body, row.Language, 0.2, sif=True)
    except:
        smry = " "
    if len(smry)<5:
        smry=" "
    summaries.append(smry)
    flatlist = []
    scores = rouge.get_scores(smry, row.Lead)[0]
    for metric in scores:
        for key in scores[metric]:
            flatlist.append(scores[metric][key])
    flatdict[i] = flatlist
    cosims.append(comparator.compare(smry, row.Lead))

In [ ]:
test_data["Summary_Fasttext_SIF"] = summaries

In [ ]:
test_data.to_pickle('./training_data/test_raw.pkl')

In [ ]:
r_scores = pd.DataFrame.from_dict(flatdict, orient="index",
                       columns=['R1_f', 'R1_p', 'R1_r', 'R2_f', 'R2_p', 'R2_r','Rl_f', 'Rl_p', 'Rl_r'])

In [ ]:
test_data = pd.merge(test_data, r_scores, left_index=True, right_index=True)

In [ ]:
test_data["cosine_sim"] = cosims

In [ ]:
test_data.head()

In [ ]:
test_data.R2_f.describe()

In [ ]:
test_data.R2_p.describe()

In [ ]:
test_data.R2_r.describe()

In [ ]:
test_data.cosine_sim.describe()